In [1]:
import tensorflow.compat.v1 as tf
import lucid_kietzmannlab.modelzoo.vision_models as models
from ipywidgets import interact, Dropdown, IntSlider
from lucid_kietzmannlab.utils import interactive_visualization, batch_visualization, plot_images
tf.compat.v1.disable_eager_execution()


In [2]:
model_checkpoint_dir="/Users/vkapoor/Downloads/models/AlexNet/training_seed_05"
training_seed = 5
model = models.AlexNetv2(model_checkpoint_dir=model_checkpoint_dir, training_seed = training_seed)


Loading model with name /Users/vkapoor/Downloads/models/AlexNet/training_seed_05/model.ckpt_epoch89
INFO:tensorflow:Restoring parameters from /Users/vkapoor/Downloads/models/AlexNet/training_seed_05/model.ckpt_epoch89


2024-06-12 15:14:45.616312: W tensorflow/core/common_runtime/graph_constructor.cc:1595] Importing a graph with a lower producer version 38 into an existing graph with producer version 1766. Shape inference will have run different parts of the graph with different producer versions.
2024-06-12 15:14:46.904371: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-12 15:14:48.102270: W tensorflow/c/c_api.cc:305] Operation '{name:'alexnet_v2/fc8/biases/Momentum/Assign' id:1952 op device:{requested: '', assigned: ''} def:{{{node alexnet_v2/fc8/biases/Momentum/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@alexnet_v2/fc8/biases"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](alexnet_v2/fc8/biases/Momentum, alexnet_v2/fc8/biases/Momentum/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either 

In [3]:
layer_shape_dict = model.layer_shape_dict
def visualize(layer_name, channel):
    interactive_visualization(model,layer_name, channel, layer_shape_dict, scope = '', channels_first = True)   

def visualize_all():
    layer_name = current_dropdown_value({'new': layer_dropdown.value})
    image_channel = batch_visualization(model, layer_name, layer_shape_dict, channel_slider, scope = '', channels_first = True)
    return image_channel              
                              

        
# Create dropdown menu for layer selection
layer_dropdown = Dropdown(options=list(layer_shape_dict.keys()), description='Layer:')

# Create slider for channel selection
channel_slider = IntSlider(min=0, max=0, description='Channel:')
        
        
def update_channel_slider(change):
    layer_name = change.new
    if layer_name in layer_shape_dict:
        
        max_channel = layer_shape_dict[layer_name][-1] - 1
        channel_slider.max = max_channel
        
        
def current_slider_value(*args):
    return channel_slider.value


def current_dropdown_value(change):
    return change['new']


channel_slider.observe(current_slider_value, names='value')
layer_dropdown.observe(current_dropdown_value, names='value')
        

       

In [4]:
layer_dropdown.observe(update_channel_slider, names='value')

# Create an interactive visualization
interact(visualize, layer_name=layer_dropdown, channel=channel_slider)     

interactive(children=(Dropdown(description='Layer:', options=('tower_0/alexnet_v2/conv1/Conv2D', 'tower_0/alex…

<function __main__.visualize(layer_name, channel)>

In [ ]:
image_channel = visualize_all()

In [ ]:

if image_channel:
    plot_images(image_channel)   

In [ ]:
layer_shape_dict.items()

In [ ]:
visualize('tower_0/alexnet_v2/conv1/Conv2D',0)

In [ ]:
graph = tf.compat.v1.get_default_graph()
for op in graph.operations:
    print(op.name)

In [ ]:

graph = tf.get_default_graph()
inputs_placeholder = graph.get_tensor_by_name('Placeholder:0')


In [ ]:
inputs_placeholder.shape

In [ ]:
labels_placeholder = graph.get_tensor_by_name('tower_0/alexnet_v2/fc8/squeezed:0')

In [ ]:
labels_placeholder.shape

In [ ]:
for op in graph.get_operations():
        try:
          if graph.get_tensor_by_name(f'{op.name}:0').shape[1] == 565: 
             print(op.name, graph.get_tensor_by_name(f'{op.name}:0').shape)
        except:
            pass     

In [ ]:
with tf.Graph().as_default() as graph:
    tf.import_graph_def(model.graph_def, name="")
    writer = tf.summary.FileWriter('AlexNetv2', graph)
    writer.close()